# Example on the MNIST Fashion Dataset

### 1. Import of the modules and the dataset

In [1]:
from neural_network import *
from keras.datasets import fashion_mnist

2023-08-30 17:02:35.602202: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2. Load the MNIST dataset

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

### 3. Build the neural network

In [3]:
net = NeuralNetwork(
    Normalization(samples=X_train),
    Reshape(output_shape=(1, 28, 28)),
    Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),
    BatchNorm2d(6),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
    BatchNorm2d(16),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Flatten(),
    Linear(in_features=400, out_features=120),
    Tanh(),
    Linear(in_features=120, out_features=84),
    Tanh(),
    Linear(in_features=84, out_features=10),
    OutputLayer(activation_function="softmax", loss_function="categorical_cross_entropy")
)
print(net)

NeuralNetwork:
 (0) Normalization(norm=255.0, dtype=float32)
 (1) Reshape(output_shape=(1, 28, 28))
 (2) Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (3) BatchNorm2d(6, eps=1e-05, momentum=0.1)
 (4) Tanh()
 (5) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (6) Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (7) BatchNorm2d(16, eps=1e-05, momentum=0.1)
 (8) Tanh()
 (9) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (10) Flatten()
 (11) Linear(in_features=400, out_features=120, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (12) Tanh()
 (13) Linear(in_features=120, out_features=84, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), ini

### 4. Choose the number of samples to train on

In [4]:
# Select N samples to train on:
N = 5000
X_train, y_train = X_train[:N], y_train[:N]
X_train.shape, y_train.shape

((5000, 28, 28), (5000,))

### 5. Train the neural network

In [5]:
net.fit(X_train, y_train, epochs=5, batch_size=64, shuffle=True)

Training on 5000 samples:
Epoch    1 of 5    	 Average Error = 0.850170 	 Average Accuracy = 70.56%                                   
Epoch    2 of 5    	 Average Error = 0.527633 	 Average Accuracy = 81.04%                                   
Epoch    3 of 5    	 Average Error = 0.458259 	 Average Accuracy = 83.74%                                   
Epoch    4 of 5    	 Average Error = 0.414455 	 Average Accuracy = 85.38%                                   
Epoch    5 of 5    	 Average Error = 0.376740 	 Average Accuracy = 86.74%                                   
Training time : 00 hours, 01 minutes, 02 seconds


### 6. Make predictions on the test samples

In [6]:
# Prediction on the 10,000 test samples:
y_pred = net.predict(X_test, to="labels")
# Shows the first 10 predicted labels vs the true labels:
y_pred[:10], y_test[:10]

(array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7]),
 array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7], dtype=uint8))

### 7. Evaluate the accuracy score

In [7]:
print(f"Accuracy score on the test set: {accuracy_score(y_test, y_pred):.2%}")

Accuracy score on the test set: 82.98%


### 8. Build a confusion matrix

In [8]:
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion matrix:\n{cm}")

Confusion matrix:
[[747   1   9  55  12   4 154   0  18   0]
 [  0 942   2  39   5   0  11   0   1   0]
 [ 13   1 631  12 187   0 141   0  15   0]
 [ 20   5   4 875  34   0  57   0   5   0]
 [  0   1  74  39 791   1  85   0   9   0]
 [  1   0   0   0   0 940   0  29   1  29]
 [124   1  85  39 151   2 571   0  27   0]
 [  0   0   0   0   0  51   0 898   0  51]
 [  2   0   6   5   5  11  16   6 948   1]
 [  0   0   0   0   0   7   0  38   0 955]]


### 9. Displays a classification report

In [9]:
class_labels = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
print(classification_report(cm, class_labels=class_labels, formatted=True))

┌───────────────────┬───────────┬───────────┬───────────┬───────────┐
│       Class       │ Precision │   Recall  │  F1-Score │  Support  │
├───────────────────┼───────────┼───────────┼───────────┼───────────┤
│    T-shirt/top    │  82.36%   │  74.70%   │  78.34%   │   1000    │
│      Trouser      │  99.05%   │  94.20%   │  96.57%   │   1000    │
│     Pullover      │  77.81%   │  63.10%   │  69.69%   │   1000    │
│       Dress       │  82.24%   │  87.50%   │  84.79%   │   1000    │
│       Coat        │  66.75%   │  79.10%   │  72.40%   │   1000    │
│      Sandal       │  92.52%   │  94.00%   │  93.25%   │   1000    │
│       Shirt       │  55.17%   │  57.10%   │  56.12%   │   1000    │
│      Sneaker      │  92.48%   │  89.80%   │  91.12%   │   1000    │
│        Bag        │  92.58%   │  94.80%   │  93.68%   │   1000    │
│    Ankle boot     │  92.18%   │  95.50%   │  93.81%   │   1000    │
└───────────────────┴───────────┴───────────┴───────────┴───────────┘
